In [1]:
from geopy.geocoders import Nominatim
import pandas as pd

#座標取得
geo_list = []
geo = pd.read_csv("geo_point.csv")
geolocator = Nominatim(user_agent="user-id")
for chimei in range(len(geo)):
    location = geolocator.geocode(geo.iloc[chimei,0]+",江東区")
    geo_list.append([location.address.split(",")[0],location.latitude, location.longitude])
df_list = pd.DataFrame(geo_list, columns=["地名","緯度","経度"])
df_list.to_csv("geo_loc_pos.csv", index=False, encoding="utf_8")

In [2]:
import folium
from branca.element import Figure
from openrouteservice import convert
import pandas as pd

line1 = pd.read_csv("geo_loc_pos.csv")

color_list = ["red","green","purple","orange","darkred","lightred","beige","darkblue","darkgreen","cadetblue","darkpurple","white","pink","lightblue","lightgreen","gray","black","lightgray","blue"]
 
points_a = []

for i in range(len(line1)):
    points_a.append([line1.iloc[i,1],line1.iloc[i,2]])

In [8]:
import folium
import openrouteservice
from branca.element import Figure
from openrouteservice import convert

key = "5b3ce3597851110001cf624804ffaeec7cd246038d01eb4d3a32f633"
client = openrouteservice.Client(key=key)

loc_place = []
for chimei in range(len(points_a)-1):
    p1 = points_a[chimei]
    p2 = points_a[chimei+1]
    p1r = tuple(reversed(p1))
    p2r = tuple(reversed(p2))

    # 経路計算 (Directions V2)
    routedict = client.directions((p1r, p2r),profile="foot-walking")
    geom = routedict["routes"][0]["geometry"]
    decoded = convert.decode_polyline(geom)
    for i in range(len(decoded["coordinates"])):
        loc_place.append(decoded["coordinates"][i])

[35.6310401, 139.7864852127717]
[35.6297628, 139.7939817]
[35.64938, 139.789728]
[35.6196145, 139.7769402]
[35.6326539, 139.7929743]


In [4]:
import csv

datalist = []

with open("geo_loc_pos.csv", encoding="utf_8") as f:
    reader = csv.reader(f)
    line = [row for row in reader]

for i in range(len(line)-2):
    p1 = float(line[i+1][1]), float(line[i+1][2])
    for j in range(i,len(line)-2):
        p2 = float(line[j+2][1]), float(line[j+2][2])
        p1r = tuple(reversed(p1))
        p2r = tuple(reversed(p2))
        mean_lat = (p1[0] + p2[0]) / 2
        mean_long = (p1[1] + p2[1]) / 2

        # 経路計算 (Directions V2)
        routedict = client.directions((p1r, p2r),profile="foot-walking")
        
        
        geom = routedict["routes"][0]["geometry"]
        decoded = convert.decode_polyline(geom)

        datalist.append([i,j+1,float(routedict["routes"][0]["summary"]["duration"])])
    
df_list = pd.DataFrame(datalist, columns=["出発地点","行先","移動時間[s]"])
df_list.to_csv("geo_loc_loc_time.csv", index=False, encoding="utf_8")

In [5]:
# Load map centred on average coordinates
ave_lat = sum(p[0] for p in points_a)/len(points_a)
ave_lon = sum(p[1] for p in points_a)/len(points_a)
fig = Figure(width=800, height=400)
my_map = folium.Map(
    location=[ave_lat, ave_lon], 
    zoom_start=13
)

#add a markers 'a'
for each in range(len(points_a)-1):
    folium.Marker(
        location=points_a[each+1],
        popup=line1.iloc[each+1,0],
        icon = folium.Icon(color=color_list[0])
    ).add_to(my_map)

# add markers 'z'
folium.Marker(
    location=[line1.iloc[0,1],line1.iloc[0,2]],
    popup=line1.iloc[0,0]
).add_to(my_map)

    
# add lines
folium.PolyLine(points_a, color=color_list[0], weight=2.5, opacity=1).add_to(my_map)
 
fig.add_child(my_map)

In [6]:
def reverse_lat_long(list_of_lat_long):
    return [(p[1], p[0]) for p in list_of_lat_long]

coord = reverse_lat_long(loc_place)
fig = Figure(width=800, height=400)

my_map1 = folium.Map(
    location=(mean_lat, mean_long),
    zoom_start=13
)

for each in range(len(points_a)-1):
    folium.Marker(
        location=points_a[each+1],
        popup=line1.iloc[each+1,0],
        icon = folium.Icon(color=color_list[0])
    ).add_to(my_map1)


folium.Marker(
    location=[line1.iloc[0,1],line1.iloc[0,2]],
    popup=line1.iloc[0,0]
).add_to(my_map1)

folium.vector_layers.PolyLine(coord, color=color_list[0], weight=2.5, opacity=1).add_to(my_map1)
fig.add_child(my_map1)